In [11]:
# connect to google drive and move to the selected directory
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Stage LIP6/CBOW"
!pwd 
!ls 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Stage LIP6/CBOW
/content/gdrive/My Drive/Stage LIP6/CBOW
data	     models.py	__pycache__  runs	 test.py
datasets.py  old	README.md    test.ipynb


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import trange
import matplotlib.pyplot as plt
import sys

import models
from datasets import CBOWDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

writer=SummaryWriter("runs/test2")

def main():
    torch.manual_seed(1)
    CUDA = torch.cuda.is_available()
    if CUDA:
        print("avaible GPUs:", torch.cuda.device_count())
        print("GPU name:", torch.cuda.get_device_name())
    print("pytorch version: ", torch.__version__)

    dataset = CBOWDataset("data/en.txt",context_size=2,max_len=10)

    # Parameters
    CONTEXT_SIZE = 2
    EMBEDDING_SIZE = 300
    EPOCHS = 5
    LEARNING_RATE = 0.01

    # Model
    model = models.CBOW(len(dataset)+2*CONTEXT_SIZE, EMBEDDING_SIZE, CONTEXT_SIZE)
    if CUDA:
        model = model.cuda()

    # Training
    losses = train(model, dataset, EPOCHS, LEARNING_RATE)
    print(losses)


def train(model, dataset, epochs, lr):
    '''
    Train a model 

    Parameters:
        model (nn.module):
        data(list of tuples):  
        words_to_idx() :dict containing a mapping word->index
    Return:
        tensor (Tensor): 
    '''
    loss_func = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    losses = []

    #### testing tensor board
    example_context,example_target=dataset[0]
    writer.add_graph(model, example_context)
    writer.close()

    for epoch in trange(epochs):
        total_loss = 0
        corrects=0
        for i in range(len(dataset)):
            context=dataset[i][0]
            target=dataset[i][1]
            output = model(context)
            loss = loss_func(output, target)

            model.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            corrects+= (predicted==target).sum().item()
        losses.append(total_loss)
        writer.add_scalar('training loss',total_loss/len(dataset),epoch*len(dataset)+i )
        writer.add_scalar('accuracy',corrects/len(dataset),epoch*len(dataset)+i )
    
    return losses


if __name__ == '__main__':
    main()


100%|██████████| 5/5 [00:00<00:00, 67.28it/s]

avaible GPUs: 1
GPU name: Tesla T4
pytorch version:  1.6.0+cu101
file:  <_io.TextIOWrapper name='data/en.txt' mode='r' encoding='utf8'>
[14.07414698600769, 6.550328969955444, 3.226123631000519, 1.5816864967346191, 0.917349174618721]


In [7]:
!python test.py

python3: can't open file 'test.py': [Errno 2] No such file or directory


In [ ]:
!tensorboard --logdir=runs

2020-08-25 05:14:14.674248: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.3.0 at http://localhost:6006/ (Press CTRL+C to quit)
